In [1]:
import pandas as pd
import requests
import os
import sqlite3
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'datalab.sqlite')
db_path

'/Users/paulcolas/code/RonanB400/decp_ml/data/datalab.sqlite'

In [21]:
conn = sqlite3.connect(db_path)
db = conn.cursor()
query = f"""
SELECT *
FROM "data.gouv.fr.2022.clean"
"""

data = pd.read_sql_query(query, conn)
data.columns

Index(['uid', 'id', 'nature', 'acheteur_id', 'acheteur_nom', 'acheteur_siren',
       'titulaire_id', 'titulaire_typeIdentifiant', 'titulaire_nom',
       'titulaire_siren', 'objet', 'montant', 'codeCPV', 'procedure',
       'dureeMois', 'dateNotification', 'datePublicationDonnees', 'formePrix',
       'attributionAvance', 'offresRecues', 'marcheInnovant', 'ccag',
       'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'tauxAvance',
       'origineUE', 'origineFrance', 'lieuExecution_code',
       'lieuExecution_typeCode', 'idAccordCadre', 'source_open_data'],
      dtype='object')

In [ ]:
acheteur_siren_df = data['acheteur_siren'].unique()
titulaire_siren_df = data['titulaire_siren'].unique()
print(len(acheteur_siren_df))
print(len(titulaire_siren_df))

10991
62841


In [55]:
echantil = acheteur_siren_df[0:100]

In [46]:
siren = '215400516'

# Clé API fournie par INSEE
api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"

# URL de l'API Sirene
url = f"https://api.insee.fr/api-sirene/3.11/siren/{siren}"

# Headers avec la clé d'API
headers = {
    "X-INSEE-Api-Key-Integration": api_key
}

# Requête GET
response = requests.get(url, headers=headers).json()

In [48]:
effectif = response['uniteLegale']['trancheEffectifsUniteLegale']
cat_entreprise = response['uniteLegale']['categorieEntreprise']
response['header']['statut']

200

In [61]:
api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"
headers = {"X-INSEE-Api-Key-Integration": api_key}
dico_dico = {}

for sir in acheteur_siren_df:
    url = f"https://api.insee.fr/api-sirene/3.11/siren/{sir}"
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            continue

        data = response.json()

        unite_legale = data.get("uniteLegale")
        if not unite_legale:
            continue

        tranche = unite_legale.get("trancheEffectifsUniteLegale")
        if not tranche:
                continue
        categorie = unite_legale.get("categorieEntreprise")
        if not categorie:
            continue

        dico_dico[sir] = [tranche, categorie]

    except Exception as e:
        continue

KeyboardInterrupt: 

In [36]:
api_key = "928d65d1-d6e4-49e1-8d65-d1d6e479e178"
headers = {"X-INSEE-Api-Key-Integration": api_key}
dico_dico = {}

for sir in acheteur_siren_df:
    url = f"https://api.insee.fr/api-sirene/3.11/siren/{sir}"
    response = requests.get(url, headers=headers).json()
    effectif_categorie = [response['uniteLegale']['trancheEffectifsUniteLegale'], response['uniteLegale']['categorieEntreprise']]
    dico_dico[sir] = effectif_categorie

KeyError: 'uniteLegale'

In [59]:
dico_dico = {}

In [5]:
dict_effectif = {
    'NN' : "Unité non-employeuse ou présumée non-employeuse (faute de déclaration reçue)",
    '00' : "0 salarié (n'ayant pas d'effectif au 31/12 mais ayant employé des salariés au cours de l'année de référence)",
    '01' : "1 ou 2 salariés",
    '02' : "3 à 5 salariés",
    '03' : "6 à 9 salariés",
    '11' : "10 à 19 salariés",
    '12' : "20 à 49 salariés",
    '21' : "50 à 99 salariés",
    '22' : "100 à 199 salariés",
    '31' : "200 à 249 salariés",
    '32' : "250 à 499 salariés",
    '41' : "500 à 999 salariés",
    '42' : "1 000 à 1 999 salariés",
    '51' : "2 000 à 4 999 salariés",
    '52' : "5 000 à 9 999 salariés",
    '53' : "10 000 salariés et plus",
    'null' : 'donnée manquante ou "sans objet"'
}

In [7]:
effectif = response['uniteLegale']['trancheEffectifsUniteLegale']
cat_entreprise = response['uniteLegale']['categorieEntreprise']

'12'